# Data Exploration for Box Office Trends in the 21st Century


This notebook and its accompanying repository attempts to answer some questions that would help guide Microsoft in making the best financial decisions possible when entering the crowded field of movie production. The contents herein are preliminary and serve as a starting point for a more in depth anaylsis of the topic and is by no means exhaustive. That being said, accuracy of findings and the data itself was an important consideration throughout the project. 

## Overview

This particular notebook focuses on data exploration and cleaning. First the data was imported and after a brief overview tables of each dataset were created to become familiar with what each file contained. Afterwards the data was processed to analyze in the following notebook. 

## Business Problem
This project covers the following hypothetical:
Microsoft wants to create a new movie studio and decided to hire me to conduct data analysis to determine what films are currently performing the best at the box office. Using multiple datasets from a number of movie databases, expected returns and risk as well as expected cost and how to best utilize recourses can be determined through anaylsis. As the newcomer to a crowded realm, further analysis was conducted to ensure a well recieved debut movie as well as to set the standard for the quality of it's future films.

## Data Understanding

Below is an image of the Schema for this collection of data I created after exploring the imported datasets.
The data has been pulled from the following sources:

[The Numbers](https://www.the-numbers.com/) <br>
[Box Office Mojo](https://www.boxofficemojo.com/) <br>
[IMDb](https://www.imdb.com/) <br>
[Rotten Tomatoes](https://www.rottentomatoes.com/) <br>
[The Movie Database](https://www.themoviedb.org/)

The Numbers and Box Office Mojo websites provided all of the financial information (budgets and revenue) needed for analysis. IMDb provids a large amount of data for writers, directors, genres, and general information for a large sum of movies. Rotten Tomatoes provides a unqiue value in terms of critical review from trained professionals. The Movie Database provides an alternative to the information available from IMDb. 


The data is described and explore in much more detail in the following section.
<img src="images/Schema.png">

# Importing and Exploring the Data 

This notebook primarily focuses importing and exploring a large amount of unfamiliar data that was available to me at the start of the project. Throughout the data exploration process, I created legends and notes to help organize and familarize myself with the data in order to process it in the next notebook. Writing out the contents of each dataset through sheer exploration helped greatly in understanding what tools I have in hand for drawing conclusions and asking appropriate questions. 

In [1]:
#import appropriate libraries
import numpy as np
import pandas as pd
import re
import sqlite3
from pandasql import sqldf

In [2]:
ls

Analysis_and_Visualizations.ipynb      cleanedData/
Data_Exploration_and_Processing.ipynb  images/
Dataset_Tables.md                      moviePresentation.pdf
README.md                              zippedData/


In [3]:
ls zippedData

bom.movie_gross.csv.gz        imdb.title.ratings.csv.gz
imdb.name.basics.csv.gz       rt.movie_info.tsv.gz
imdb.title.akas.csv.gz        rt.reviews.tsv.gz
imdb.title.basics.csv.gz      tmdb.movies.csv.gz
imdb.title.crew.csv.gz        tn.movie_budgets.csv.gz
imdb.title.principals.csv.gz


### Box Office Mojo Dataset Initial Exploration


This dataset contains box-office revenue (both domestic and foreign) and the year the movie was released by title and studio. The movies were released between the years 2010 and 2018 and the set contains thousands of entries. Because we are only trying to determine current trends, this collection of movies released in the 2010s (2019 excluded) is a good fit for our goal.  



[Dataset Table](https://github.com/ddey117/Micosoft_Movie_Analysis/blob/master/Dataset_Tables.md)
Please refer to the Box Office Mojo section of the above link for my own descritpion of each element of this dataset.

In [4]:
# Import the Box Office Mojo data and review some general information

df_bom = pd.read_csv('zippedData/bom.movie_gross.csv.gz', compression='gzip', header=0)
df_bom.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3387 entries, 0 to 3386
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   title           3387 non-null   object 
 1   studio          3382 non-null   object 
 2   domestic_gross  3359 non-null   float64
 3   foreign_gross   2037 non-null   object 
 4   year            3387 non-null   int64  
dtypes: float64(1), int64(1), object(3)
memory usage: 132.4+ KB


In [5]:
#looking at the table above, we can see that foreign_gross is a different data type than domestic_gross

In [6]:
df_bom.head(10) #glance at beginning of dataset 

,title,studio,domestic_gross,foreign_gross,year
0,Toy Story 3,BV,415000000.0,652000000,2010
1,Alice in Wonderland (2010),BV,334200000.0,691300000,2010
2,Harry Potter and the Deathly Hallows Part 1,WB,296000000.0,664300000,2010
3,Inception,WB,292600000.0,535700000,2010
4,Shrek Forever After,P/DW,238700000.0,513900000,2010
5,The Twilight Saga: Eclipse,Sum.,300500000.0,398000000,2010
6,Iron Man 2,Par.,312400000.0,311500000,2010
7,Tangled,BV,200800000.0,391000000,2010
8,Despicable Me,Uni.,251500000.0,291600000,2010
9,How to Train Your Dragon,P/DW,217600000.0,277300000,2010


In [7]:
#it seems that float64 is the correct data type for these columns
#passing the parameter thosuands= "," quickly corrects this mismatch in datatypes


df_bom = pd.read_csv('zippedData/bom.movie_gross.csv.gz', compression='gzip', header=0, thousands=",")
df_bom.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3387 entries, 0 to 3386
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   title           3387 non-null   object 
 1   studio          3382 non-null   object 
 2   domestic_gross  3359 non-null   float64
 3   foreign_gross   2037 non-null   float64
 4   year            3387 non-null   int64  
dtypes: float64(2), int64(1), object(2)
memory usage: 132.4+ KB


### The Numbers Dataset  Initial Exploration

The Numbers Dataset contains very useful information for the cost and revenue of a large number of films from 1915 to 2020. I have added a few more columns to look at each month and year seperately, as well as engineered another column to represent the total return on investment (ROI) of each film. See code below for more details.

[Dataset Table](https://github.com/ddey117/Micosoft_Movie_Analysis/blob/master/Dataset_Tables.md)
Please refer to the Numbers section of the above link for my own descritpion of each element of this dataset.

In [8]:
df_tn = pd.read_csv('zippedData/tn.movie_budgets.csv.gz', compression='gzip', header=0)

In [9]:
display(df_tn.tail(10))
df_tn.info()

,id,release_date,movie,production_budget,domestic_gross,worldwide_gross
5772,73,"Jan 13, 2012",Newlyweds,"$9,000","$4,584","$4,584"
5773,74,"Feb 26, 1993",El Mariachi,"$7,000","$2,040,920","$2,041,928"
5774,75,"Oct 8, 2004",Primer,"$7,000","$424,760","$841,926"
5775,76,"May 26, 2006",Cavite,"$7,000","$70,071","$71,644"
5776,77,"Dec 31, 2004",The Mongol King,"$7,000",$900,$900
5777,78,"Dec 31, 2018",Red 11,"$7,000",$0,$0
5778,79,"Apr 2, 1999",Following,"$6,000","$48,482","$240,495"
5779,80,"Jul 13, 2005",Return to the Land of Wonders,"$5,000","$1,338","$1,338"
5780,81,"Sep 29, 2015",A Plague So Pleasant,"$1,400",$0,$0
5781,82,"Aug 5, 2005",My Date With Drew,"$1,100","$181,041","$181,041"


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5782 entries, 0 to 5781
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   id                 5782 non-null   int64 
 1   release_date       5782 non-null   object
 2   movie              5782 non-null   object
 3   production_budget  5782 non-null   object
 4   domestic_gross     5782 non-null   object
 5   worldwide_gross    5782 non-null   object
dtypes: int64(1), object(5)
memory usage: 271.2+ KB


Although I only managed to succesfully merge 1322 rows to include studio data, that is still three times as much information involving studios that I was able to compare to revenue data inside of the Rotten Tomatoes dataset. The dataset is looking good and ready to start working with for some analysis. 

### IMBd Dataset Initial Exploration

[Dataset Table](https://github.com/ddey117/Micosoft_Movie_Analysis/blob/master/Dataset_Tables.md)
Please refer to the IMVd section of the above link for my own descritpion of each element of this relation database.


[IMDb weighted average for ratings](https://help.imdb.com/article/imdb/track-movies-tv/weighted-average-ratings/GWT2DSBYVT2F25SK?ref_=helpart_nav_8#)


It seems that another column *ordering* must be used when there are duplicate tconst or nconst values to aid with indexing 

In [10]:
df_imdb_names = pd.read_csv('zippedData/imdb.name.basics.csv.gz', compression='gzip', header=0)
df_imdb_titles = pd.read_csv('zippedData/imdb.title.basics.csv.gz', compression='gzip', header=0)
df_imdb_alt_titles = pd.read_csv('zippedData/imdb.title.akas.csv.gz', compression='gzip', header=0)
df_imdb_crew = pd.read_csv('zippedData/imdb.title.crew.csv.gz', compression='gzip', header=0)
df_imdb_principals = pd.read_csv('zippedData/imdb.title.principals.csv.gz', compression='gzip', header=0)
df_imdb_ratings = pd.read_csv('zippedData/imdb.title.ratings.csv.gz', compression='gzip', header=0)

In [11]:
print('Table of unique name identifiers for movie industry personnel.')
display(df_imdb_names.head())
display(df_imdb_names.info())
print('Table of unique identifiers for film titles and metadata for those films.')
display(df_imdb_titles.head())
display(df_imdb_titles.info())
print(f'The {df_imdb_titles.keys()[-1]} series of this dataframe is a comma seperated string of values.')

print('Alternative titles for movie releases.')
display(df_imdb_alt_titles.head())
display(df_imdb_alt_titles.info())
print('Table of unqie identifierd for films and their associated directors and writers.')
display(df_imdb_crew.head())
display(df_imdb_crew.info())
print('table of unique identifiers of films and a list of unqie identifiers of names.')
print('contains number of entries(rows) to describe individuals who worked on a particular film.')
display(df_imdb_principals.head())
display(df_imdb_principals.info())
print('Table for relating films to IMDb community ratings.')
display(df_imdb_ratings.head())
df_imdb_ratings.info()

Table of unique name identifiers for movie industry personnel.


,nconst,primary_name,birth_year,death_year,primary_profession,known_for_titles
0,nm0061671,Mary Ellen Bauder,NaN,NaN,"miscellaneous,production_manager,producer","tt0837562,tt2398241,tt0844471,tt0118553"
1,nm0061865,Joseph Bauer,NaN,NaN,"composer,music_department,sound_department","tt0896534,tt6791238,tt0287072,tt1682940"
2,nm0062070,Bruce Baum,NaN,NaN,"miscellaneous,actor,writer","tt1470654,tt0363631,tt0104030,tt0102898"
3,nm0062195,Axel Baumann,NaN,NaN,"camera_department,cinematographer,art_department","tt0114371,tt2004304,tt1618448,tt1224387"
4,nm0062798,Pete Baxter,NaN,NaN,"production_designer,art_department,set_decorator","tt0452644,tt0452692,tt3458030,tt2178256"


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 606648 entries, 0 to 606647
Data columns (total 6 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   nconst              606648 non-null  object 
 1   primary_name        606648 non-null  object 
 2   birth_year          82736 non-null   float64
 3   death_year          6783 non-null    float64
 4   primary_profession  555308 non-null  object 
 5   known_for_titles    576444 non-null  object 
dtypes: float64(2), object(4)
memory usage: 27.8+ MB


None

Table of unique identifiers for film titles and metadata for those films.


...

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1028186 entries, 0 to 1028185
Data columns (total 6 columns):
 #   Column      Non-Null Count    Dtype 
---  ------      --------------    ----- 
 0   tconst      1028186 non-null  object
 1   ordering    1028186 non-null  int64 
 2   nconst      1028186 non-null  object
 3   category    1028186 non-null  object
 4   job         177684 non-null   object
 5   characters  393360 non-null   object
dtypes: int64(1), object(5)
memory usage: 47.1+ MB


None

Table for relating films to IMDb community ratings.


,tconst,averagerating,numvotes
0,tt10356526,8.3,31
1,tt10384606,8.9,559
2,tt1042974,6.4,20
3,tt1043726,4.2,50352
4,tt1060240,6.5,21


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 73856 entries, 0 to 73855
Data columns (total 3 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   tconst         73856 non-null  object 
 1   averagerating  73856 non-null  float64
 2   numvotes       73856 non-null  int64  
dtypes: float64(1), int64(1), object(1)
memory usage: 1.7+ MB


### The Movie Database Initial Exploration

The Movie Database is an open alternative to IMDb with an easily accesible API. It is not as old as IMBd and therefore is less complete/established. 

[TMDb Popularity Explained](https://developers.themoviedb.org/3/getting-started/popularity)


[Dataset Table](https://github.com/ddey117/Micosoft_Movie_Analysis/blob/master/Dataset_Tables.md)
Please reference The Movie Database section of the above table for my description of each element of this dataset.


In [12]:
df_tmdb = pd.read_csv('zippedData/tmdb.movies.csv.gz', compression='gzip', header=0)

In [13]:
df_tmdb.drop('Unnamed: 0', axis=1, inplace=True)

display(df_tmdb.info())
df_tmdb.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26517 entries, 0 to 26516
Data columns (total 9 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   genre_ids          26517 non-null  object 
 1   id                 26517 non-null  int64  
 2   original_language  26517 non-null  object 
 3   original_title     26517 non-null  object 
 4   popularity         26517 non-null  float64
 5   release_date       26517 non-null  object 
 6   title              26517 non-null  object 
 7   vote_average       26517 non-null  float64
 8   vote_count         26517 non-null  int64  
dtypes: float64(2), int64(2), object(5)
memory usage: 1.8+ MB


None

,genre_ids,id,original_language,original_title,popularity,release_date,title,vote_average,vote_count
0,"[12, 14, 10751]",12444,en,Harry Potter and the Deathly Hallows: Part 1,33.533,2010-11-19,Harry Potter and the Deathly Hallows: Part 1,7.7,10788
1,"[14, 12, 16, 10751]",10191,en,How to Train Your Dragon,28.734,2010-03-26,How to Train Your Dragon,7.7,7610
2,"[12, 28, 878]",10138,en,Iron Man 2,28.515,2010-05-07,Iron Man 2,6.8,12368
3,"[16, 35, 10751]",862,en,Toy Story,28.005,1995-11-22,Toy Story,7.9,10174
4,"[28, 878, 12]",27205,en,Inception,27.920,2010-07-16,Inception,8.3,22186


In [14]:
df_tmdb.duplicated(subset='id').value_counts() #see if duplicates at primary key ('id' column)

False    25497
True      1020
dtype: int64

In [15]:
df_tmdb = df_tmdb.drop_duplicates(subset='id')

In [16]:
df_tmdb.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 25497 entries, 0 to 26516
Data columns (total 9 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   genre_ids          25497 non-null  object 
 1   id                 25497 non-null  int64  
 2   original_language  25497 non-null  object 
 3   original_title     25497 non-null  object 
 4   popularity         25497 non-null  float64
 5   release_date       25497 non-null  object 
 6   title              25497 non-null  object 
 7   vote_average       25497 non-null  float64
 8   vote_count         25497 non-null  int64  
dtypes: float64(2), int64(2), object(5)
memory usage: 1.9+ MB


### Rotten Tomatoes Dataset  Initial Exploration

[Dataset Table](https://github.com/ddey117/Micosoft_Movie_Analysis/blob/master/Dataset_Tables.md)
Please refer to the Rotten Tomatoes section of the above link for my own descritpion of each element of this dataset.


In [17]:
# Importing the tsv files from Rotten Tomatoes 
df_rt = pd.read_csv('zippedData/rt.movie_info.tsv.gz', compression='gzip', 
                    header=0, sep='\t', encoding= 'unicode_escape')
df_rt_reviews = pd.read_csv('zippedData/rt.reviews.tsv.gz', compression='gzip', 
                            header=0, sep='\t', encoding= 'unicode_escape')
display(df_rt.info())
display(df_rt.head())

display(df_rt_reviews.info())
df_rt_reviews.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1560 entries, 0 to 1559
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   id            1560 non-null   int64 
 1   synopsis      1498 non-null   object
 2   rating        1557 non-null   object
 3   genre         1552 non-null   object
 4   director      1361 non-null   object
 5   writer        1111 non-null   object
 6   theater_date  1201 non-null   object
 7   dvd_date      1201 non-null   object
 8   currency      340 non-null    object
 9   box_office    340 non-null    object
 10  runtime       1530 non-null   object
 11  studio        494 non-null    object
dtypes: int64(1), object(11)
memory usage: 146.4+ KB


None

,id,synopsis,rating,genre,director,writer,theater_date,dvd_date,currency,box_office,runtime,studio
0,1,"This gritty, fast-paced, and innovative police...",R,Action and Adventure|Classics|Drama,William Friedkin,Ernest Tidyman,"Oct 9, 1971","Sep 25, 2001",NaN,NaN,104 minutes,NaN
1,3,"New York City, not-too-distant-future: Eric Pa...",R,Drama|Science Fiction and Fantasy,David Cronenberg,David Cronenberg|Don DeLillo,"Aug 17, 2012","Jan 1, 2013",$,"600,000",108 minutes,Entertainment One
2,5,Illeana Douglas delivers a superb performance ...,R,Drama|Musical and Performing Arts,Allison Anders,Allison Anders,"Sep 13, 1996","Apr 18, 2000",NaN,NaN,116 minutes,NaN
3,6,Michael Douglas runs afoul of a treacherous su...,R,Drama|Mystery and Suspense,Barry Levinson,Paul Attanasio|Michael Crichton,"Dec 9, 1994","Aug 27, 1997",NaN,NaN,128 minutes,NaN
4,7,NaN,NR,Drama|Romance,Rodney Bennett,Giles Cooper,NaN,NaN,NaN,NaN,200 minutes,NaN


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 54432 entries, 0 to 54431
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   id          54432 non-null  int64 
 1   review      48869 non-null  object
 2   rating      40915 non-null  object
 3   fresh       54432 non-null  object
 4   critic      51710 non-null  object
 5   top_critic  54432 non-null  int64 
 6   publisher   54123 non-null  object
 7   date        54432 non-null  object
dtypes: int64(2), object(6)
memory usage: 3.3+ MB


None

,id,review,rating,fresh,critic,top_critic,publisher,date
0,3,A distinctly gallows take on contemporary fina...,3/5,fresh,PJ Nabarro,0,Patrick Nabarro,"November 10, 2018"
1,3,It's an allegory in search of a meaning that n...,NaN,rotten,Annalee Newitz,0,io9.com,"May 23, 2018"
2,3,... life lived in a bubble in financial dealin...,NaN,fresh,Sean Axmaker,0,Stream on Demand,"January 4, 2018"
3,3,Continuing along a line introduced in last yea...,NaN,fresh,Daniel Kasman,0,MUBI,"November 16, 2017"
4,3,... a perverse twist on neorealism...,NaN,fresh,NaN,0,Cinema Scope,"October 12, 2017"


In [18]:
#convert release_date values to datetime format
df_rt_reviews['date'] = pd.to_datetime(df_rt_reviews['date'])

The Rotten Tomatoes dataset does not contain a column to easily link to the other datasets. It may be possible to do some webscrapping to compile a list of titles based on the synopsis of the movie as well as the theater release dates and DVD release dates. However, I decided against this due to time constraints. The dataset does include some box_office revenue data, so I decided to only join within the two Rotten Tomato datasets. This is the only dataset that has information dedidcated to what professional critics directly have to say about certain films.  

In [19]:
df_rt = df_rt.dropna(subset=['currency', 'box_office', 'studio'])

In [20]:
df_rt.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 305 entries, 1 to 1555
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   id            305 non-null    int64 
 1   synopsis      305 non-null    object
 2   rating        305 non-null    object
 3   genre         305 non-null    object
 4   director      268 non-null    object
 5   writer        247 non-null    object
 6   theater_date  301 non-null    object
 7   dvd_date      301 non-null    object
 8   currency      305 non-null    object
 9   box_office    305 non-null    object
 10  runtime       304 non-null    object
 11  studio        305 non-null    object
dtypes: int64(1), object(11)
memory usage: 31.0+ KB


In [21]:
df_rt_reviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 54432 entries, 0 to 54431
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   id          54432 non-null  int64         
 1   review      48869 non-null  object        
 2   rating      40915 non-null  object        
 3   fresh       54432 non-null  object        
 4   critic      51710 non-null  object        
 5   top_critic  54432 non-null  int64         
 6   publisher   54123 non-null  object        
 7   date        54432 non-null  datetime64[ns]
dtypes: datetime64[ns](1), int64(2), object(5)
memory usage: 3.3+ MB


## Data Preparation

### Data Cleaning and Processing for Budget Information

In [22]:
df_tn.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5782 entries, 0 to 5781
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   id                 5782 non-null   int64 
 1   release_date       5782 non-null   object
 2   movie              5782 non-null   object
 3   production_budget  5782 non-null   object
 4   domestic_gross     5782 non-null   object
 5   worldwide_gross    5782 non-null   object
dtypes: int64(1), object(5)
memory usage: 271.2+ KB


Above we can see from calling the info() method that we have some datatypes to change for The Numbers dataframe. Creating a function to run for production_budget, domestic_gross, and worldwide_gross to clean the data and change the datatype will prove useful for later analysis.

In [23]:
def string_to_int(dataframe, col):
    """Used in order to remove problematic characters from strings in dataframes.
    
    This function removes '$' and ',' from a string series in a dataframe and returns 
    the dataframe with cleaned series as an int datatype. 
    
    Parameters:
    dataframe: pandas dataframe 
    col (object, int): column of dataframe to clean and convert to int 
    
    Returns:
    dataframe with col arg converted to int type and '$' and ',' characters removed. 
    
      """
    dataframe[col] = dataframe[col].str.replace('$', '').str.replace(',', '').astype(int)
    return dataframe

In [24]:
help(string_to_int)

Help on function string_to_int in module __main__:

string_to_int(dataframe, col)
    Used in order to remove problematic characters from strings in dataframes.
    
    This function removes '$' and ',' from a string series in a dataframe and returns 
    the dataframe with cleaned series as an int datatype. 
    
    Parameters:
    dataframe: pandas dataframe 
    col (object, int): column of dataframe to clean and convert to int 
    
    Returns:
    dataframe with col arg converted to int type and '$' and ',' characters removed.



In [25]:
#clean three different columns of interest
string_to_int(df_tn, "production_budget")
string_to_int(df_tn, "domestic_gross")
string_to_int(df_tn, "worldwide_gross")
df_tn.head()

,id,release_date,movie,production_budget,domestic_gross,worldwide_gross
0,1,"Dec 18, 2009",Avatar,425000000,760507625,2776345279
1,2,"May 20, 2011",Pirates of the Caribbean: On Stranger Tides,410600000,241063875,1045663875
2,3,"Jun 7, 2019",Dark Phoenix,350000000,42762350,149762350
3,4,"May 1, 2015",Avengers: Age of Ultron,330600000,459005868,1403013963
4,5,"Dec 15, 2017",Star Wars Ep. VIII: The Last Jedi,317000000,620181382,1316721747


In [26]:
#convert release_date values to datetime format
df_tn['release_date'] = pd.to_datetime(df_tn['release_date']) 
display(df_tn['release_date'].head())
df_tn.info()

0   2009-12-18
1   2011-05-20
2   2019-06-07
3   2015-05-01
4   2017-12-15
Name: release_date, dtype: datetime64[ns]

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5782 entries, 0 to 5781
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   id                 5782 non-null   int64         
 1   release_date       5782 non-null   datetime64[ns]
 2   movie              5782 non-null   object        
 3   production_budget  5782 non-null   int64         
 4   domestic_gross     5782 non-null   int64         
 5   worldwide_gross    5782 non-null   int64         
dtypes: datetime64[ns](1), int64(4), object(1)
memory usage: 271.2+ KB


In [27]:
#create a new column for the year the movie was released
#create a new colum for the month the movie was released
df_tn['release_year'] = pd.DatetimeIndex(df_tn['release_date']).year
df_tn['release_month'] = pd.DatetimeIndex(df_tn['release_date']).month
display(df_tn.head())
df_tn.info()

,id,release_date,movie,production_budget,domestic_gross,worldwide_gross,release_year,release_month
0,1,2009-12-18,Avatar,425000000,760507625,2776345279,2009,12
1,2,2011-05-20,Pirates of the Caribbean: On Stranger Tides,410600000,241063875,1045663875,2011,5
2,3,2019-06-07,Dark Phoenix,350000000,42762350,149762350,2019,6
3,4,2015-05-01,Avengers: Age of Ultron,330600000,459005868,1403013963,2015,5
4,5,2017-12-15,Star Wars Ep. VIII: The Last Jedi,317000000,620181382,1316721747,2017,12


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5782 entries, 0 to 5781
Data columns (total 8 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   id                 5782 non-null   int64         
 1   release_date       5782 non-null   datetime64[ns]
 2   movie              5782 non-null   object        
 3   production_budget  5782 non-null   int64         
 4   domestic_gross     5782 non-null   int64         
 5   worldwide_gross    5782 non-null   int64         
 6   release_year       5782 non-null   int64         
 7   release_month      5782 non-null   int64         
dtypes: datetime64[ns](1), int64(6), object(1)
memory usage: 361.5+ KB


In [28]:
display(df_tn['release_year'].min())
df_tn['release_year'].max()

1915

2020

In order to analyize current trends, it is important to only incude data relevent to the current century. Another important feature to consider for the budget data is Return on Investment,or ROI. This is a very important metric for considering the financial success of a given movie in the box office. Therefore, this new feature, ROI, will display the total return on the initial investment by meeans of the films budget and global revenue. This value is rounded to the nearest hundreth place for ease of viewing and to mantain acceptable accuracy of the data. I also decided to add another series to the dataframe to show the net revenue as opposed to just a percentage. In general, a percent return in terms of initial investment is an invaluable tool for determining financial success of any buisness decision. However, net revenue is also important for determining the exact magniutde of any financial gain. 

In [29]:
df_tn['worldwide_gross']
ROI_tn = round((((df_tn['worldwide_gross'] - df_tn['production_budget'] ) / df_tn['production_budget']) * 100), 2)
netRev_tn = round((df_tn['worldwide_gross'] - df_tn['production_budget']), 2)
df_tn['ROI'] = ROI_tn
df_tn['net_revenue'] = netRev_tn

display(df_tn.head())
df_tn.info()

,id,release_date,movie,production_budget,domestic_gross,worldwide_gross,release_year,release_month,ROI,net_revenue
0,1,2009-12-18,Avatar,425000000,760507625,2776345279,2009,12,553.26,2351345279
1,2,2011-05-20,Pirates of the Caribbean: On Stranger Tides,410600000,241063875,1045663875,2011,5,154.67,635063875
2,3,2019-06-07,Dark Phoenix,350000000,42762350,149762350,2019,6,-57.21,-200237650
3,4,2015-05-01,Avengers: Age of Ultron,330600000,459005868,1403013963,2015,5,324.38,1072413963
4,5,2017-12-15,Star Wars Ep. VIII: The Last Jedi,317000000,620181382,1316721747,2017,12,315.37,999721747


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5782 entries, 0 to 5781
Data columns (total 10 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   id                 5782 non-null   int64         
 1   release_date       5782 non-null   datetime64[ns]
 2   movie              5782 non-null   object        
 3   production_budget  5782 non-null   int64         
 4   domestic_gross     5782 non-null   int64         
 5   worldwide_gross    5782 non-null   int64         
 6   release_year       5782 non-null   int64         
 7   release_month      5782 non-null   int64         
 8   ROI                5782 non-null   float64       
 9   net_revenue        5782 non-null   int64         
dtypes: datetime64[ns](1), float64(1), int64(7), object(1)
memory usage: 451.8+ KB


In [30]:
display(df_tn.loc[df_tn['ROI'] == -100].head())#checking suspicious data
df_tn.loc[df_tn['worldwide_gross'] == 0].head() #confirming assumptions

,id,release_date,movie,production_budget,domestic_gross,worldwide_gross,release_year,release_month,ROI,net_revenue
194,95,2020-12-31,Moonfall,150000000,0,0,2020,12,-100.0,-150000000
479,80,2017-12-13,Bright,90000000,0,0,2017,12,-100.0,-90000000
480,81,2019-12-31,Army of the Dead,90000000,0,0,2019,12,-100.0,-90000000
535,36,2020-02-21,Call of the Wild,82000000,0,0,2020,2,-100.0,-82000000
670,71,2019-08-30,PLAYMOBIL,75000000,0,0,2019,8,-100.0,-75000000


,id,release_date,movie,production_budget,domestic_gross,worldwide_gross,release_year,release_month,ROI,net_revenue
194,95,2020-12-31,Moonfall,150000000,0,0,2020,12,-100.0,-150000000
479,80,2017-12-13,Bright,90000000,0,0,2017,12,-100.0,-90000000
480,81,2019-12-31,Army of the Dead,90000000,0,0,2019,12,-100.0,-90000000
535,36,2020-02-21,Call of the Wild,82000000,0,0,2020,2,-100.0,-82000000
670,71,2019-08-30,PLAYMOBIL,75000000,0,0,2019,8,-100.0,-75000000


In [31]:
# Drop ROI == -100 as proxy for zero values
df_tn.reset_index(inplace=True)
df_tn_drop = df_tn.loc[df_tn['ROI'] == -100].index
df_tn.drop(df_tn_drop, axis = 0, inplace=True)
df_tn.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5414 entries, 0 to 5781
Data columns (total 11 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   index              5414 non-null   int64         
 1   id                 5414 non-null   int64         
 2   release_date       5414 non-null   datetime64[ns]
 3   movie              5414 non-null   object        
 4   production_budget  5414 non-null   int64         
 5   domestic_gross     5414 non-null   int64         
 6   worldwide_gross    5414 non-null   int64         
 7   release_year       5414 non-null   int64         
 8   release_month      5414 non-null   int64         
 9   ROI                5414 non-null   float64       
 10  net_revenue        5414 non-null   int64         
dtypes: datetime64[ns](1), float64(1), int64(8), object(1)
memory usage: 507.6+ KB


In [32]:
df_tn.drop('index', axis=1, inplace=True)

In [33]:
df_tn['movie'].duplicated().value_counts()

False    5338
True       76
Name: movie, dtype: int64

In [34]:
df_tn[df_tn.duplicated(keep=False, subset='movie')].sort_values(by='movie')

,id,release_date,movie,production_budget,domestic_gross,worldwide_gross,release_year,release_month,ROI,net_revenue
5614,15,1916-12-24,"20,000 Leagues Under the Sea",200000,8000000,8000000,1916,12,3900.00,7800000
4270,71,1954-12-23,"20,000 Leagues Under the Sea",5000000,28200000,28200000,1954,12,464.00,23200000
5016,17,1984-11-09,A Nightmare on Elm Street,1800000,25504513,25504513,1984,11,1316.92,23704513
1648,49,2010-04-30,A Nightmare on Elm Street,35000000,63075011,117729621,2010,4,236.37,82729621
2032,33,1992-11-11,Aladdin,28000000,217350219,504050219,1992,11,1700.18,476050219
...,...,...,...,...,...,...,...,...,...,...
3403,4,2017-04-21,Unforgettable,12000000,11368012,16221211,2017,4,35.18,4221211
4583,84,2006-11-03,Unknown,3700000,26403,3355048,2006,11,-9.32,-344952
1420,21,2011-02-18,Unknown,40000000,63686397,136123083,2011,2,240.31,96123083
2313,14,2005-09-16,Venom,25000000,881745,881745,2005,9,-96.47,-24118255


These "duplicate" movies actually seem to be different movies that share the same title. I am going to keep all of the movie titles. I am ready to save this a clean csv after limiting movies to current century.

In [35]:
df_tn = df_tn[df_tn['release_year'] >= 2000]

In [36]:
df_tn['release_year'].value_counts()   #check if filter for dates worked (current century)

2010    258
2006    253
2008    249
2015    241
2011    241
2009    218
2005    218
2013    212
2007    210
2002    209
2014    207
2012    207
2016    204
2004    202
2003    196
2000    189
2001    179
2017    162
2018    140
2019     49
Name: release_year, dtype: int64

In [37]:
# Write data to a CSV file 
df_tn.to_csv('cleanedData/cleanedTN.csv', index=False) 

The Numbers dataset is now cleaned and ready for conducting analysis or as preliminary starting point for joining with seperate datasets. Below is the process that was conducted for the Box Office Mojo dataset before merging with The Numbers data.

In [38]:
df_bom.describe() #glance at summary statistics, including the range of years the dataset encapsulates. 

,domestic_gross,foreign_gross,year
count,3.359000e+03,2.037000e+03,3387.000000
mean,2.874585e+07,7.487281e+07,2013.958075
std,6.698250e+07,1.374106e+08,2.478141
min,1.000000e+02,6.000000e+02,2010.000000
25%,1.200000e+05,3.700000e+06,2012.000000
50%,1.400000e+06,1.870000e+07,2014.000000
75%,2.790000e+07,7.490000e+07,2016.000000
max,9.367000e+08,9.605000e+08,2018.000000


In [39]:
df_bom.isna().sum()  #check for missing data

title                0
studio               5
domestic_gross      28
foreign_gross     1350
year                 0
dtype: int64

In [40]:
print('Percentage of Null foreign_gross Values:', len(df_bom[df_bom.foreign_gross.isna()])/ len(df_bom))
print('Number of Unique Cforeign_gross Values:', df_bom.foreign_gross.nunique())

Percentage of Null foreign_gross Values: 0.3985828166519043
Number of Unique Cforeign_gross Values: 1204


Although SQL is not necessary to get this information, it is just personal prefference to perform the query in the way shown below.

In [41]:
 #lambda function to write sql queries  
pysqldf = lambda q: sqldf(q, globals())

In [42]:
qNa1 = """SELECT title, foreign_gross, domestic_gross 
        FROM df_bom
        WHERE foreign_gross IS NULL
        ; """

In [43]:
pysqldf(qNa1)

,title,foreign_gross,domestic_gross
0,Flipped,None,1800000.0
1,The Polar Express (IMAX re-issue 2010),None,673000.0
2,Tiny Furniture,None,392000.0
3,Grease (Sing-a-Long re-issue),None,366000.0
4,Last Train Home,None,288000.0
...,...,...,...
1345,The Quake,None,6200.0
1346,Edward II (2018 re-release),None,4800.0
1347,El Pacto,None,2500.0
1348,The Swan,None,2400.0


Checking a few of these titles to see what the foreign_gross should be, I came across this:
[Quake](https://www.boxofficemojo.com/title/tt6523720/?ref_=bo_se_r_1) 

A foreign_gross of $13,831,059 for the movie titled "The Quake" is significantly different than the domestic gross of 6200. It is clear that treating the None values as zero would have a noticable impact on the data, especially when almost 40\% of the data is null. I have decided to only look at domestic_gross for this dataset and drop the foreign gross column.


In [44]:
df_bom.drop(columns = 'foreign_gross', inplace=True)

In [45]:
df_bom.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3387 entries, 0 to 3386
Data columns (total 4 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   title           3387 non-null   object 
 1   studio          3382 non-null   object 
 2   domestic_gross  3359 non-null   float64
 3   year            3387 non-null   int64  
dtypes: float64(1), int64(1), object(2)
memory usage: 106.0+ KB


In [46]:
df_bom.dropna(inplace=True)   #drop all remaining null values

In [47]:
df_bom.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3356 entries, 0 to 3386
Data columns (total 4 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   title           3356 non-null   object 
 1   studio          3356 non-null   object 
 2   domestic_gross  3356 non-null   float64
 3   year            3356 non-null   int64  
dtypes: float64(1), int64(1), object(2)
memory usage: 131.1+ KB


In [48]:
# Write data to a CSV file for speperate data analysis
df_bom.to_csv('cleanedData/cleanedBOM.csv', index=False) 

Looking at the two different datasets involving revenue (BOM and TN), I decided to make The Numbers dataset the primary source of  budget information because it not only contains more data but it also already contains information to determine the cost of films with the column for movie budgets. Thus, I believe when joining the two dataframes it would be best to do a left join onto The Numbers dataframe. The Numbers column also already has the ROI column and net revenue loaded from before. 

*Including the date in the title section for the BOM df in paranthesis might cause some issues when trying to join the tables.* 

Below is my process for removing the paranthesis to ensure the data merges with minimal errors. 

In [49]:
#check number of titles that end with parantheses 

df_bom_titles = df_bom['title']
df_tn_titles = df_tn['movie']
print(f"df_bom titles ending in parentheses: {df_bom_titles.str.endswith(')').sum()}")
print(f"df_tn titles ending in parentheses: {df_tn_titles.str.endswith(')').sum()}")

df_bom titles ending in parentheses: 320
df_tn titles ending in parentheses: 7


In [50]:
pattern = '\s+\(\d{4}\)'  #expression to grab 4 digits in parantheses AND white space before it  
df_bom['title'].replace(pattern, '', regex = True, inplace = True)

Even though I am just beginning to learn the tools available to a data scientist, I was able to solve this regex with little experience due to the following user friendly website:
[Regex solver](https://regex101.com/)

I recommend all new programmers to check it out and play around!

In [51]:
display(df_bom.head(30)) #check if regex was succesfull
df_tn.info()

,title,studio,domestic_gross,year
0,Toy Story 3,BV,415000000.0,2010
1,Alice in Wonderland,BV,334200000.0,2010
2,Harry Potter and the Deathly Hallows Part 1,WB,296000000.0,2010
3,Inception,WB,292600000.0,2010
4,Shrek Forever After,P/DW,238700000.0,2010
5,The Twilight Saga: Eclipse,Sum.,300500000.0,2010
6,Iron Man 2,Par.,312400000.0,2010
7,Tangled,BV,200800000.0,2010
8,Despicable Me,Uni.,251500000.0,2010
9,How to Train Your Dragon,P/DW,217600000.0,2010


<class 'pandas.core.frame.DataFrame'>
Int64Index: 4044 entries, 0 to 5781
Data columns (total 10 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   id                 4044 non-null   int64         
 1   release_date       4044 non-null   datetime64[ns]
 2   movie              4044 non-null   object        
 3   production_budget  4044 non-null   int64         
 4   domestic_gross     4044 non-null   int64         
 5   worldwide_gross    4044 non-null   int64         
 6   release_year       4044 non-null   int64         
 7   release_month      4044 non-null   int64         
 8   ROI                4044 non-null   float64       
 9   net_revenue        4044 non-null   int64         
dtypes: datetime64[ns](1), float64(1), int64(7), object(1)
memory usage: 347.5+ KB


The two budget datasets are finally ready to be merged together.

In [52]:
# merge dataframes to have revenue data that includes studios
df_tn_bom = df_tn.merge(df_bom, left_on = ['movie',  'release_year'], 
                        right_on= ['title', 'year'], how= 'left')

In [53]:
display(df_tn_bom.head())
df_tn_bom.info()

,id,release_date,movie,production_budget,domestic_gross_x,worldwide_gross,release_year,release_month,ROI,net_revenue,title,studio,domestic_gross_y,year
0,1,2009-12-18,Avatar,425000000,760507625,2776345279,2009,12,553.26,2351345279,NaN,NaN,NaN,NaN
1,2,2011-05-20,Pirates of the Caribbean: On Stranger Tides,410600000,241063875,1045663875,2011,5,154.67,635063875,Pirates of the Caribbean: On Stranger Tides,BV,241100000.0,2011.0
2,3,2019-06-07,Dark Phoenix,350000000,42762350,149762350,2019,6,-57.21,-200237650,NaN,NaN,NaN,NaN
3,4,2015-05-01,Avengers: Age of Ultron,330600000,459005868,1403013963,2015,5,324.38,1072413963,Avengers: Age of Ultron,BV,459000000.0,2015.0
4,5,2017-12-15,Star Wars Ep. VIII: The Last Jedi,317000000,620181382,1316721747,2017,12,315.37,999721747,NaN,NaN,NaN,NaN


<class 'pandas.core.frame.DataFrame'>
Int64Index: 4044 entries, 0 to 4043
Data columns (total 14 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   id                 4044 non-null   int64         
 1   release_date       4044 non-null   datetime64[ns]
 2   movie              4044 non-null   object        
 3   production_budget  4044 non-null   int64         
 4   domestic_gross_x   4044 non-null   int64         
 5   worldwide_gross    4044 non-null   int64         
 6   release_year       4044 non-null   int64         
 7   release_month      4044 non-null   int64         
 8   ROI                4044 non-null   float64       
 9   net_revenue        4044 non-null   int64         
 10  title              1322 non-null   object        
 11  studio             1322 non-null   object        
 12  domestic_gross_y   1322 non-null   float64       
 13  year               1322 non-null   float64       
dtypes: datet

In [54]:
df_tn_bom.drop(['title', 'domestic_gross_x', 'domestic_gross_y', 'year'], axis= 1, inplace = True)

In [55]:
# Write data to a CSV file 
# Nndex=False, do not want it included in our output
df_tn_bom.to_csv('cleanedData/mergedBudgetTables.csv', index=False) 

### Data Cleaning and Processing for IMDb Dataset

The IMDb dataset is a relational database with easily identifiable keys. Therefore, it would make since to handle this data set with some SQL code.

I have included a screenshot of the section of the IMBd ratings FAQ from their website to help make my own top 250 list of movies.

<img src="images/weighted_ratings.png">

[More Information for IMBd Ratings](https://help.imdb.com/article/imdb/track-movies-tv/ratings-faq/G67Y87TFYYP6TWAV#)


In [56]:
#SQL query used to grab all averageratings for IMDb dataset

qnum = """SELECT averagerating
        FROM df_imdb_titles t
        JOIN df_imdb_ratings r
            USING(tconst)
        ;"""

In [57]:
df_numvotes = pysqldf(qnum)

In [58]:
avg_vote = pysqldf(qnum).mean()  #find the mean vote for dataset

In [59]:
print(avg_vote) #R in formula
df_numvotes.describe()

averagerating    6.332729
dtype: float64


,averagerating
count,73856.000000
mean,6.332729
std,1.474978
min,1.000000
25%,5.500000
50%,6.500000
75%,7.400000
max,10.000000


The SQL query below will be used to load a database for analysis of the top movies based on the weighted rating using the formula given on the IMDb website. The titles and ratings tables were joined using the primary key of tconst. The query also filtered the results to only show movies with over 25000 votes, as per the IMDb website for their top 250 movie list. Furthermore, the results were filtered to only include data for the 21st century (2000 inclusive). 

In [60]:
qtop = """SELECT  tconst,primary_title, averagerating,
                numvotes, start_year, 
                runtime_minutes, genres,
        (numvotes*1.0 / (numvotes*1.0 + 25000)) * averagerating
        + (25000 / (numvotes*1.0 + 25000)) * 6.332729 AS weighted_rating
        FROM df_imdb_titles t
        JOIN df_imdb_ratings r
            USING(tconst)
        WHERE numvotes > 25000 AND start_year >= 2000
        ORDER BY weighted_rating DESC
        ;"""

In [61]:
df_weighted_movies = pysqldf(qtop)

In [62]:
#this dataframe contains all movies filtered by vote count as per IMDb rules for their top movie set
#it is also filtered to only include movies in the 21st century
#Filtering by vote count helps improve the accuracy of the rated rating feature I created
df_weighted_movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1623 entries, 0 to 1622
Data columns (total 8 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   tconst           1623 non-null   object 
 1   primary_title    1623 non-null   object 
 2   averagerating    1623 non-null   float64
 3   numvotes         1623 non-null   int64  
 4   start_year       1623 non-null   int64  
 5   runtime_minutes  1623 non-null   float64
 6   genres           1623 non-null   object 
 7   weighted_rating  1623 non-null   float64
dtypes: float64(3), int64(2), object(3)
memory usage: 101.6+ KB


In [63]:
# Write data to a CSV file 
df_weighted_movies.to_csv('cleanedData/IMDb_weighted.csv', index=False) 

In [64]:
df_top_250 = df_weighted_movies.loc[:250]
display(df_top_250.describe())
df_top_250.head()

,averagerating,numvotes,start_year,runtime_minutes,weighted_rating
count,251.000000,2.510000e+02,251.000000,251.000000,251.000000
mean,7.888048,3.081599e+05,2013.876494,125.458167,7.651921
std,0.330237,2.666205e+05,2.511710,25.349265,0.293005
min,7.400000,2.674300e+04,2010.000000,83.000000,7.287477
25%,7.600000,9.363600e+04,2012.000000,106.000000,7.431190
50%,7.800000,2.492450e+05,2014.000000,122.000000,7.605942
75%,8.100000,4.398780e+05,2016.000000,138.500000,7.821582
max,9.300000,1.841066e+06,2019.000000,321.000000,8.766946


,tconst,primary_title,averagerating,numvotes,start_year,runtime_minutes,genres,weighted_rating
0,tt1375666,Inception,8.8,1841066,2010,148.0,"Action,Adventure,Sci-Fi",8.766946
1,tt5813916,The Mountain II,9.3,100568,2016,135.0,"Action,Drama,War",8.709230
2,tt4154796,Avengers: Endgame,8.8,441135,2019,181.0,"Action,Adventure,Sci-Fi",8.667674
3,tt0816692,Interstellar,8.6,1299334,2014,169.0,"Adventure,Drama,Sci-Fi",8.557200
4,tt1675434,The Intouchables,8.5,677343,2011,112.0,"Biography,Comedy,Drama",8.422856


In [65]:
df_top_250.to_csv('cleanedData/top250df.csv', index=False) #export csv file for analysis in seperate notebook

At this point the table is ready to be anaylzed in terms of runtime for the top rated movies. However, further action is required in order to access the strings of genres in the dataframe. After trying for probably too long to avoid a for loop, I actually ended up going the opposite way and used a nested for loop in a for loop to first create a list of genres by caling the .split() method and then iterating through that list to create a dictionary. The goal was to get a count of each genre in order to display a bar graph of what genres comprise the top 250 movies of the current century. 

In [66]:
top_s = df_top_250.loc[:, 'genres']

In [67]:
genre_count_dict = {}   #initialize the dictionary needed for the genre counts
for genres in top_s:
    genre_list = genres.split(',')    #create a list from the strings in the series
    for genre in genre_list:            #iterate through the newly created list to 
        if genre in genre_count_dict:    #make a dictionary of value counts
            genre_count_dict[genre] += 1   
        else:
            genre_count_dict[genre] = 1    

In [68]:
print(genre_count_dict)  #success

{'Action': 79, 'Adventure': 66, 'Sci-Fi': 29, 'Drama': 160, 'War': 8, 'Biography': 40, 'Comedy': 58, 'Music': 8, 'Thriller': 46, 'Western': 2, 'Animation': 22, 'Crime': 37, 'Documentary': 11, 'Sport': 8, 'Fantasy': 15, 'Mystery': 23, 'History': 13, 'Romance': 20, 'Family': 7, 'Horror': 7, 'Musical': 2}


In [70]:
#convert dictionary back to pandas dataframe
df_genre_counts = pd.DataFrame.from_dict(genre_count_dict, orient="index") 

In [71]:
display(df_genre_counts.head())
type(df_genre_counts)

,0
Action,79
Adventure,66
Sci-Fi,29
Drama,160
War,8


pandas.core.frame.DataFrame

In [72]:
df_genre_counts.reset_index(inplace=True) #reset index

In [73]:
df_genre_counts.keys()  #check keys to clean up column names

Index(['index', 0], dtype='object')

In [74]:
df_genre_counts.rename(columns = {'index': 'genre', 0: 'genre_count'}, inplace=True) #rename columns
df_genre_counts.keys()  #recheck dataframe keys

Index(['genre', 'genre_count'], dtype='object')

In [75]:
df_genre_counts.to_csv('cleanedData/genreCounts', index=False) #export csv file for analysis in seperate notebook

In [76]:
#query to create dataframe to collect the nconst of the directors of the top 250 movies

qt3 = """SELECT primary_title, weighted_rating, nconst, category    
        FROM df_top_250 t
        JOIN df_imdb_principals p
            USING(tconst)
            WHERE category = 'director'
        ORDER BY weighted_rating DESC
        ;"""

In [77]:
top_directors = pysqldf(qt3)   #create dataframe of codes (nconst) for top directors

In [78]:
top_directors.head(10)

,primary_title,weighted_rating,nconst,category
0,Inception,8.766946,nm0634240,director
1,The Mountain II,8.709230,nm3809021,director
2,Avengers: Endgame,8.667674,nm0751577,director
3,Avengers: Endgame,8.667674,nm0751648,director
4,Interstellar,8.557200,nm0634240,director
5,The Intouchables,8.422856,nm0619923,director
6,The Intouchables,8.422856,nm0865918,director
7,Avengers: Infinity War,8.422144,nm0751577,director
8,Avengers: Infinity War,8.422144,nm0751648,director
9,Whiplash,8.415594,nm3227090,director


In [79]:
#query to select nconst of the writers invovled in top 250 movies

qt4 = """SELECT primary_title, weighted_rating, nconst, category    
        FROM df_top_250 t
        JOIN df_imdb_principals p
            USING(tconst)
            WHERE category = 'writer'
        ORDER BY weighted_rating DESC
        ;"""

In [80]:
top_writers = pysqldf(qt4) #dataframe to collect codes (nconst) of top 250 writers

In [81]:
top_writers.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 487 entries, 0 to 486
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   primary_title    487 non-null    object 
 1   weighted_rating  487 non-null    float64
 2   nconst           487 non-null    object 
 3   category         487 non-null    object 
dtypes: float64(1), object(3)
memory usage: 15.3+ KB


In [82]:
#query to convert nconst into names of directors that worked on top 250 movies
qt5 = """SELECT primary_name, primary_title, weighted_rating    
        FROM top_directors d
        JOIN df_imdb_names n
            USING(nconst)
        ORDER BY weighted_rating DESC
        ;"""

In [83]:
df_top_250_directors = pysqldf(qt5)

In [84]:
#query to convert nconst into names of writers that worked on top 250 movies
qt6 = """SELECT primary_name, primary_title, weighted_rating    
        FROM top_writers d
        JOIN df_imdb_names n
            USING(nconst)
        ORDER BY weighted_rating DESC
        ;"""

In [85]:
df_top_250_writers = pysqldf(qt6)

In [86]:
display(df_top_250_directors.head())
df_top_250_writers.head()

,primary_name,primary_title,weighted_rating
0,Christopher Nolan,Inception,8.766946
1,Alper Caglar,The Mountain II,8.709230
2,Anthony Russo,Avengers: Endgame,8.667674
3,Joe Russo,Avengers: Endgame,8.667674
4,Christopher Nolan,Interstellar,8.557200


,primary_name,primary_title,weighted_rating
0,Jack Kirby,Avengers: Endgame,8.667674
1,Christopher Markus,Avengers: Endgame,8.667674
2,Stephen McFeely,Avengers: Endgame,8.667674
3,Stan Lee,Avengers: Endgame,8.667674
4,Jonathan Nolan,Interstellar,8.557200


In [87]:
df_top_250_directors.to_csv('cleanedData/top250_directors.csv', index=False) #export csv file for analysis in seperate notebook
df_top_250_writers.to_csv('cleanedData/top250_writers.csv', index=False)

End of data exploration and initial processing. Data Analysis is carried out on the following Notebook. 